<a href="https://colab.research.google.com/github/CAP4770-group9-project/final-project/blob/notebook%2Fkmeans/k-means-clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# K-Means Clustering for Stock Portfolio Diversification

Diversification is a very important step in stock portfolio creation. Picking several stocks rather than just one helps to reduce some of the inherent risk with investing. However, to reap the full benefits of diversification one not only has to pick more than one stock but also ensure that the behaviour of the stocks are not correlated.

This measure of "differentiality" in stock return and variance behavior can be calculated using K means clustering. The stock price data over diffrent periods of time can be put into the clustering algorithm with the goal of clustering the stock based on diffrent behavior. Stocks from these clusters then can be chosen in order to create a well-diversified portfolio.

## Data preprocessing

### Import data and calculate parameters

Now we can import the S&P500 data into Colab

In [273]:
#change directory to where the data is located 
!pwd
%cd drive/My Drive/cap4770-project

/content/drive/My Drive/cap4770-project
[Errno 2] No such file or directory: 'drive/My Drive/cap4770-project'
/content/drive/My Drive/cap4770-project


In [274]:
import pandas as pd
import time

#use timer to get process times
start_time = time.time()

#read csv
data = pd.read_csv("SP_DAILY_2000-2020.csv")
print("read data --- %s seconds ---" % (time.time() - start_time))

availableStocks = data['Symbol'].unique()
print("Number of stocks in the dataset: ", len(availableStocks))
print(data)

read data --- 1.6911883354187012 seconds ---
Number of stocks in the dataset:  503
        Symbol        Date    Open   Close     High     Low    Volume
0          MMM  2020-03-26  131.79  135.98  136.355  130.61   5476351
1          MMM  2020-03-25  133.15  131.54  134.690  126.80   7732467
2          MMM  2020-03-24  122.29  132.72  133.450  121.00   9304832
3          MMM  2020-03-23  128.16  117.87  128.400  114.04   7920348
4          MMM  2020-03-20  138.07  124.89  139.240  122.71   9582251
...        ...         ...     ...     ...      ...     ...       ...
2250258    ZTS  2013-02-07   31.00   32.00   32.730   31.00   3800800
2250259    ZTS  2013-02-06   30.98   31.03   31.430   30.75   2126100
2250260    ZTS  2013-02-05   31.25   31.04   31.980   30.85   5013200
2250261    ZTS  2013-02-04   31.09   31.02   31.990   30.76   7695400
2250262    ZTS  2013-02-01   31.50   31.01   31.740   30.47  66789100

[2250263 rows x 7 columns]


Now we can extract the training data from the dataset

In [0]:
#create DateTime column
stocksData = data

stocksData['DateTime'] = pd.to_datetime(stocksData.apply(lambda row: row['Date'], axis=1))

print(stocksData)

In [0]:
#get full training data set
trainData = stocksData[(stocksData['DateTime'] > '2010-03-26') & (stocksData['DateTime'] < '2015-03-26')]

#get training data set for 2015
trainData2015 = stocksData[(stocksData['DateTime'] > '2014-03-26') & (stocksData['DateTime'] < '2015-03-26')]
print(trainData2015)

We can perform an initial visualization of the data by plotting

In [0]:
%matplotlib notebook
%matplotlib inline


import matplotlib
import matplotlib.pyplot as plt


In [0]:
grouped_day_stocks = []
grouped_day = []

#plot all data
#plt.figure()
#for i in range(0, len(availableStocks)):
#  grouped_day_stocks.append(stocksData[stocksData['Symbol'] == availableStocks[i]])
#  grouped_day.append(grouped_day_stocks[i].groupby(lambda x: grouped_day_stocks[i]['DateTime'][x]))
#for j in range(0, len(availableStocks)):
#  plt.plot(grouped_day[j].size().index, grouped_day_stocks[j]['Close'])
#plt.show()

In [0]:
#group day for all training data
grouped_day_stocks = []
grouped_day = []

for i in range(0, len(availableStocks)):
  grouped_day_stocks.append(trainData[trainData['Symbol'] == availableStocks[i]])
  grouped_day.append(grouped_day_stocks[i].groupby(lambda x: grouped_day_stocks[i]['DateTime'][x]))

#print("grouped day stocks")
#print(grouped_day_stocks, "\n")



In [0]:
#print(grouped_day_stocks)

In [0]:
#plot only training data
#plt.figure()
#for j in range(0, len(availableStocks)):
#  plt.plot(grouped_day[j].size().index, grouped_day_stocks[j]['Close'])
#plt.show()

For this 1 year sample of the training data, we can calculate the annualized average returns (based on price at close), annualized average volatility (based on price at close), the average intraday price variation (pct change between daily high and low), average diffrence in price between open and close and annualized trading volume from the data. 

The intraday price variation and diffrence between open and close will be used as additional measures of volatility and hopefully will add richness to the resutls of the k-means analysis

In [0]:
#group day for 2015 data
grouped_day_stocks = []
grouped_day = []

for i in range(0, len(availableStocks)):
  grouped_day_stocks.append(trainData2015[trainData2015['Symbol'] == availableStocks[i]])
  grouped_day.append(grouped_day_stocks[i].groupby(lambda x: grouped_day_stocks[i]['DateTime'][x]))

  

In [0]:
print(len(grouped_day_stocks))

In [0]:
#plot performance of the SP500 over training period

!pip install ffn
import ffn
prices = ffn.get('^GSPC',start='2014-03-26', end='2015-03-26')
ax1 = prices.plot(title="Performance of S&P500 from 2014-2015", figsize=(15, 10))

In [0]:
#plot index overall returns
returns = prices.to_returns().dropna()
print(returns)
print(returns.mean() *252)
ax = returns.hist(figsize=(10,10))

In [0]:
"""
Need to rebase price data in order to get a clearer visualization

"""

#plot only training data 2015
#plt.figure()
#for j in range(0, len(availableStocks)):
#  plt.plot(grouped_day[j].size().index, grouped_day_stocks[j]['Close'])
#plt.show()

In [0]:
annualized_data = pd.DataFrame(index=availableStocks, columns=['Returns', 'Volatility','Volume', 'HighLowDiff', 'OpenCloseDiff'])

"""
Returns seem to be off a bit, as the sp500 itself has positive average returns but the returns data is negative

"""
#aggregate annualized stock data used in cluserting analysis into dataframe
for i in range(0, len(grouped_day_stocks)):
  
  annualized_data.loc[availableStocks[i]]['Returns'] = grouped_day_stocks[i]['Close'].pct_change().mean() * 252
  annualized_data.loc[availableStocks[i]]['Volatility'] = grouped_day_stocks[i]['Close'].pct_change().std() * 252
  annualized_data.loc[availableStocks[i]]['Volume'] = grouped_day_stocks[i]['Volume'].mean() * 252
  annualized_data.loc[availableStocks[i]]['HighLowDiff'] = ((grouped_day_stocks[i]['High'] - grouped_day_stocks[i]['Low']) / grouped_day_stocks[i]['Low']).mean()
  annualized_data.loc[availableStocks[i]]['OpenCloseDiff'] = ((grouped_day_stocks[i]['Close'] - grouped_day_stocks[i]['Open']) / grouped_day_stocks[i]['Open']).mean()  

print("Annualized data for each stock in S&P500 calculated form 2014 to 2015:")
print(annualized_data)

print("Min/Max statistics: \n")
print("Max returns: ")
print(annualized_data[annualized_data['Returns'] == max(annualized_data['Returns'])], "\n")
print("Min returns: ")
print(annualized_data[annualized_data['Returns'] == min(annualized_data['Returns'])], "\n")

print("Max volatility: ")
print(annualized_data[annualized_data['Volatility'] == max(annualized_data['Volatility'],key=abs)], "\n")
print("Min volatility: ")
print(annualized_data[annualized_data['Volatility'] == min(annualized_data['Volatility'],key=abs)], "\n")

print("Max volume: ")
print(annualized_data[annualized_data['Volume'] == max(annualized_data['Volume'],key=abs)], "\n")
print("Min volume: ")
print(annualized_data[annualized_data['Volume'] == min(annualized_data['Volume'],key=abs)], "\n")

print("Max HighLowDiff: ")
print(annualized_data[annualized_data['HighLowDiff'] == max(annualized_data['HighLowDiff'],key=abs)], "\n")
print("Min HighLowDiff: ")
print(annualized_data[annualized_data['HighLowDiff'] == min(annualized_data['HighLowDiff'],key=abs)], "\n")

print("Max OpenCloseDiff: ")
print(annualized_data[annualized_data['OpenCloseDiff'] == max(annualized_data['OpenCloseDiff'],key=abs)], "\n")
print("Min OpenCloseDiff: ")
print(annualized_data[annualized_data['OpenCloseDiff'] == min(annualized_data['OpenCloseDiff'],key=abs)], "\n")

#

###Plotting

####4D Plot | Returns, Volatility, Volume, OpenCloseDiff

We will now attempt to plot the returns, volatility, volume and openclose diff attributes

In [0]:
#Concatenating the returns and variances and volume into a single data-frame
ret_var_vol_ocDiff = annualized_data.drop(['HighLowDiff'], axis=1)
print(ret_var_vol_ocDiff)

In [0]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

#the below graph plots returns, volatility and volume on the axes and uses a heat map to display OpenCloseDiff
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(111, projection='3d')
fig.suptitle('Annualized S&P 500 Data 2014-2015', fontsize=16)

x = list(ret_var_vol_ocDiff['Returns'].dropna().values)
y = list(ret_var_vol_ocDiff['Volatility'].dropna().values)
z = list(ret_var_vol_ocDiff['Volume'].dropna().values)
c = list(ret_var_vol_ocDiff['OpenCloseDiff'].dropna().values)

img = ax.scatter(x, y, z, c=c, cmap=plt.hot())
fig.colorbar(img)
ax.set_xlabel('Returns')
ax.set_ylabel('Volatility')
ax.set_zlabel('Volume')
ax.set_xlim(-1,1)
ax.set_zlim(0,.5e10)
ax.set_ylim(0,25)

plt.show()

For now we will run the K means using only 3 attributes, the mean returns, variance, and volume

In [0]:
#Concatenating the returns and variances and volume into a single data-frame
ret_var_vol = annualized_data.drop(['HighLowDiff','OpenCloseDiff'], axis=1)
print(ret_var_vol)

####3D Plot | Returns, Volatility, Volume

In [0]:
%matplotlib inline
#first plot data to spot outliers
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

x = list(ret_var_vol['Returns'].dropna().values)
y = list(ret_var_vol['Volatility'].dropna().values)
z = list(ret_var_vol['Volume'].dropna().values)
#print(x, "\n", len(x))
#print(y, "\n", len(y))
#print(z, "\n", len(z))

ax.scatter(x, y, z, c='r', marker='o')

ax.set_xlabel('Returns')
ax.set_ylabel('Volatility')
ax.set_zlabel('Volume')

plt.show()

In [0]:
#identify the volatility outlier, drop, then graph again
#2014-2015 volatility outliers 'AAPL', 'V', 'HBI'
#outlier = ret_var_vol['Volatility'] == max(ret_var_vol['Volatility'],key=abs)
#outlierIndex = ret_var_vol[outlier].index
#print(outlierIndex, "\n")

outlierIndex = ['AAPL','V','HBI']
ret_var_vol.drop(outlierIndex, inplace =True)

In [0]:
#code to drop volume outliers
#2014 to 2015 volume outlier 'BAC'
#outlier = ret_var_vol['Volume'] == max(ret_var_vol['Volume'],key=abs)
#outlierIndex = ret_var_vol[outlier].index
#print(outlierIndex, "\n")

outlierIndex = 'BAC'
ret_var_vol.drop(outlierIndex, inplace =True)

In [0]:
#regraph after outliers are dropped

fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(111, projection='3d')
fig.suptitle('Annualized S&P 500 Data 2014-2015', fontsize=16)

x = list(ret_var_vol['Returns'].dropna().values)
y = list(ret_var_vol['Volatility'].dropna().values)
z = list(ret_var_vol['Volume'].dropna().values)
#print(x, "\n", len(x))
#print(y, "\n", len(y))
#print(z, "\n", len(z))

ax.scatter(x, y, z, c='r', marker='o')
#ax.title("Annualized S&P500 Data")

ax.set_xlabel('Returns')
ax.set_ylabel('Volatility')
ax.set_zlabel('Volume')

plt.show()

##Using PCA to Reduce Dimensionality

The dimensionality of the data chosen for the analysis below is arbitrary. The most common measures of a stock's performance in portfolio optimization analysis are mean and variance. However, since the data obtained from the AlphaVantage API includes additional dimensionality, it is worth exploring if the additional dimensions would add richness to our analysis.

Before we perform our clustering analysis, we can determine the most prominent features in the data using a Principal Component Analysis. This will be used to reduce the 5 dimensional data into a smaller amount of principal components. The clustering analysis can then be performed

First the data needs to be standardized as PCA is affected by scale. Normalizing will improve the maximization of the variance for each component that is needed in PCA.

In [0]:
#https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60
#https://towardsdatascience.com/an-approach-to-choosing-the-number-of-components-in-a-principal-component-analysis-pca-3b9f3d6e73fe
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

#use imputer to remobe NaN values and replace with mean along each column
imp = SimpleImputer(strategy="mean")
scale = StandardScaler()
print(annualized_data)

#use scaler to normalize data before clustering
annualized_data_scaled = scale.fit_transform(imp.fit_transform(annualized_data.values))
print(annualized_data_scaled)

Now we can calculate the explained variance for each attribute

In [0]:
#Fitting the PCA algorithm with our Data
pca = PCA().fit(annualized_data_scaled)

#Plotting the Cumulative Summation of the Explained Variance
plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') #for each component
plt.title('Stock Dataset Explained Variance')
plt.show()

This indicates that there are only 3 components the indicate almost 100% of the variance in the model, therefore implying that there are only 3 principal components that need to be calculated. We can now proceed to perform the PCA analysis.

In [0]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
principalComponents = pca.fit_transform(annualized_data_scaled)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2', 'principal component 3'])
print(principalDf)

Now we can concatenate the tickers onto the principal components dataframe

In [0]:
tickers = pd.DataFrame(annualized_data.index, columns=['Tickers'])
labeled_pca_df = pd.concat([tickers, principalDf],axis = 1).dropna()
print(labeled_pca_df)

In [0]:
#drop outliers
#outliers seem to have high principal component 3 value
#outlier = labeled_PCA_df['principal component 3'] == max(labeled_PCA_df['principal component 3'],key=abs)
#outlierTicker = labeled_PCA_df[outlier]['Tickers']
#outlierIndex = labeled_PCA_df[outlier].index
#print(outlierTicker)
#print(outlierIndex, "\n")

outlierTickers = ['AAPL', 'BAC', 'HBI', 'V']
print(labeled_pca_df)
outlierIndex = labeled_pca_df[labeled_pca_df['Tickers'].isin(outlierTickers)].index
labeled_pca_df.drop(outlierIndex, inplace =True)
print(labeled_pca_df)

Now we can plot the principal components in principal component space

In [0]:
fig = plt.figure(figsize = (15,15))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('Principal Component 3', fontsize = 15)
ax.set_ylabel('Principal Component 1', fontsize = 15)
ax.set_zlabel('Principal Component 2', fontsize = 15)
ax.set_title('3 component PCA', fontsize = 20)

x = list(labeled_pca_df['principal component 3'].dropna().values)
y = list(labeled_pca_df['principal component 1'].dropna().values)
z = list(labeled_pca_df['principal component 2'].dropna().values)

ax.scatter(x, y, z, c='r', marker='o')


plt.show()

## Clustering Analysis | PCA Data

Now that the data has been converted from a 5d to a 3d space, we can perform the clustering analysis

In [0]:
from pylab import plot,show
from numpy import vstack,array
from numpy.random import rand
import numpy as np
from scipy.cluster.vq import kmeans,vq
import pandas as pd
import pandas_datareader as dr
from math import sqrt
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt

In [0]:
X = labeled_pca_df.drop('Tickers', axis=1).values

In [0]:
#generate elbow curve
distorsions = []
for k in range(2, 20):
    k_means = KMeans(n_clusters=k)
    k_means.fit(X)
    distorsions.append(k_means.inertia_)

fig = plt.figure(figsize=(15, 5))
plt.plot(range(2, 20), distorsions)
plt.grid(True)
plt.title('Elbow curve')

In [0]:
import  pylab as pl

#compute K means with n = 8 and get centroids
kmeans = KMeans(n_clusters = 8)
kmeans.fit(X)
centroids = kmeans.cluster_centers_
#print(centroids)
#print(kmeans.labels_)
fig = plt.figure(figsize = (15,15))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('Principal Component 3', fontsize = 15)
ax.set_ylabel('Principal Component 1', fontsize = 15)
ax.set_zlabel('Principal Component 2', fontsize = 15)
ax.set_title('K means clustering (n=8) on 3 component PCA Stock Data', fontsize = 20)

x = list(labeled_pca_df['principal component 3'].dropna().values)
y = list(labeled_pca_df['principal component 1'].dropna().values)
z = list(labeled_pca_df['principal component 2'].dropna().values)


ax.scatter(x, y, z, c=k_means.labels_, marker='o')
plot(centroids[:,0],centroids[:,1],centroids[:,2],'sb',markersize=8)
ax.legend(['Centroids'])
ax.set_xlabel('Principal Component 3')
ax.set_ylabel('Principal Component 1')
ax.set_zlabel('Principal Component 2')
ax.set_xlim(-6,4)

plt.show()

In [0]:
fig = plt.figure(figsize=(15,15))
fig.suptitle('K-Means(n=8) Clustered PCA, 3 vs. 1', fontsize=16)
ax = fig.add_subplot(2, 1, 1)
ax.scatter(x, y, c=k_means.labels_, marker='o')
plot(centroids[:,0],centroids[:,1], 'sb',markersize=8)
ax.legend(['Centroids'])
ax.set_xlabel('PCA 3')
ax.set_ylabel('PCA 1')

plt.show()

In [0]:
fig = plt.figure(figsize=(15,15))
fig.suptitle('K-Means(n=8) Clustered PCA, 3 vs. 2', fontsize=16)
ax = fig.add_subplot(2, 1, 1)
ax.scatter(z, x, c=k_means.labels_, marker='o')
plot(centroids[:,0],centroids[:,1], 'sb',markersize=8)
ax.legend(['Centroids'])
ax.set_xlabel('PCA 2')
ax.set_ylabel('PCA 3')

plt.show()

In [0]:
fig = plt.figure(figsize=(15,15))
fig.suptitle('K-Means(n=8) Clustered PCA, 2 vs. 1', fontsize=16)
ax = fig.add_subplot(2, 1, 1)
ax.scatter(y, z, c=k_means.labels_, marker='o')
plot(centroids[:,0],centroids[:,1], 'sb',markersize=8)
ax.legend(['Centroids'])
ax.set_xlabel('PCA 1')
ax.set_ylabel('PCA 2')

plt.show()

In [0]:
cluster_labels = pd.DataFrame(kmeans.labels_,columns=['Cluster'])
labeled_stocks_pca = cluster_labels.set_index(labeled_pca_df['Tickers'])
print(labeled_stocks_pca)

##Clustering Analysis | Returns, Volume, Volatility

In [0]:
#taken from: https://towardsdatascience.com/machine-learning-for-stock-clustering-using-k-means-algorithm-126bc1ace4e1
#https://www.pythonforfinance.net/2018/02/08/stock-clusters-using-k-means-algorithm-in-python/

from pylab import plot,show
from numpy import vstack,array
from numpy.random import rand
import numpy as np
from scipy.cluster.vq import kmeans,vq
import pandas as pd
import pandas_datareader as dr
from math import sqrt
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt



In order to ensure accurate clustering results, we can now scale the variance and volume data

In [0]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

#use imputer to remobe NaN values and replace with mean along each column
imp = SimpleImputer(strategy="mean")
scale = StandardScaler()
print(ret_var_vol)
unscaled_ret_var_vol = ret_var_vol
#use scaler to normalize data before clustering
X = scale.fit_transform(imp.fit_transform(ret_var_vol.values))
print(X)

In [0]:
ret_var_vol = pd.DataFrame(data = X
             , columns = ['Returns', 'Volatility', 'Volume'])
print(ret_var_vol)

In [0]:
#generate elbow curve
distorsions = []
for k in range(2, 20):
    k_means = KMeans(n_clusters=k)
    k_means.fit(X)
    distorsions.append(k_means.inertia_)

fig = plt.figure(figsize=(15, 5))
plt.plot(range(2, 20), distorsions)
plt.grid(True)
plt.title('Elbow curve')

Now we can undertake in the K-means clustering and plot the results

In [0]:
import  pylab as pl

#compute K means with n = 8 and get centroids
kmeans = KMeans(n_clusters = 8)
kmeans.fit(X)
centroids = kmeans.cluster_centers_
#print(centroids)
#print(kmeans.labels_)

fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(111, projection='3d')
fig.suptitle('K-Means(n=5) Clustered S&P 500 Data 2014-2015', fontsize=16)

x = list(ret_var_vol['Returns'].values)
y = list(ret_var_vol['Volatility'].values)
z = list(ret_var_vol['Volume'].values)

ax.scatter(x, y, z, c=k_means.labels_, marker='o')
plot(centroids[:,0],centroids[:,1],centroids[:,2],'sb',markersize=8)
ax.legend(['Centroids'])
ax.set_xlabel('Returns')
ax.set_ylabel('Volatility')
ax.set_zlabel('Volume')

plt.show()



In [0]:
fig = plt.figure(figsize=(15,15))
fig.suptitle('K-Means(n=5) Clustered S&P 500 Data 2014-2015, Volatility vs. Returns', fontsize=16)
ax = fig.add_subplot(2, 1, 1)
ax.scatter(x, y, c=k_means.labels_, marker='o')
plot(centroids[:,0],centroids[:,1], 'sb',markersize=8)
ax.legend(['Centroids'])
ax.set_xlabel('Returns')
ax.set_ylabel('Volatility')

plt.show()

In [0]:
fig = plt.figure(figsize=(15,15))
fig.suptitle('K-Means(n=5) Clustered S&P 500 Data 2014-2015, Volatility vs. Volume', fontsize=16)
ax = fig.add_subplot(2, 1, 1)
ax.scatter(y, z, c=k_means.labels_, marker='o')
plot(centroids[:,1],centroids[:,2], 'sb',markersize=8)
ax.legend(['Centroids'])
ax.set_xlabel('Volatility')
ax.set_ylabel('Volume')

plt.show()

In [0]:
fig = plt.figure(figsize=(15,15))
fig.suptitle('K-Means(n=5) Clustered S&P 500 Data 2014-2015, Returns vs. Volume', fontsize=16)
ax = fig.add_subplot(2, 1, 1)
ax.scatter(x, z, c=k_means.labels_, marker='o')
plot(centroids[:,0],centroids[:,2], 'sb',markersize=8)
ax.legend(['Centroids'])
ax.set_xlabel('Returns')
ax.set_ylabel('Volume')

plt.show()

We try diffrent axes configurations to get additional perspectives of the data

In [0]:
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(111, projection='3d')
fig.suptitle('K-Means(n=5) Clustered S&P 500 Data 2014-2015', fontsize=16)

ax.scatter(z, y, x, c=k_means.labels_, marker='o')
plot(centroids[:,0],centroids[:,1],centroids[:,2],'sb',markersize=8)
ax.legend(['Centroids'])
ax.set_xlabel('Volume')
ax.set_ylabel('Volatility')
ax.set_zlabel('Returns')

plt.show()

In [0]:
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(111, projection='3d')
fig.suptitle('K-Means(n=5) Clustered S&P 500 Data 2014-2015', fontsize=16)


ax.scatter(z, x, y, c=k_means.labels_, marker='o')
plot(centroids[:,0],centroids[:,1],centroids[:,2],'sb',markersize=8)
ax.legend(['Centroids'])
ax.set_xlabel('Volume')
ax.set_ylabel('Returns')
ax.set_zlabel('Volatility')

plt.show()

Now that the clustering has been completed, we can create a new dataframe composed on the ticker names and their cluster labels.

In [0]:
tickers = pd.DataFrame(ret_var_vol.index, columns=['Tickers'])
cluster_labels = pd.DataFrame(kmeans.labels_,columns=['Cluster'])
labeled_stocks_3d = pd.concat([tickers, cluster_labels],axis = 1).dropna()
print(labeled_stocks_3d)

## Clustering Analysis | Returns, Volatility

A second clustering analysis can be done with just the returns and volatility parameters. The results of clustering on imporvement of diversification from this analysis and the three parameter analysis can be later compared.

In [0]:
#generate new dataframe with only two parameters
print(ret_vol_var)
ret_var = ret_vol_var.drop(['Volume'], axis=1)
print(ret_var)

####2D Plot

In [0]:
#first plot data 
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(2 ,1 ,1)
fig.suptitle('S&P 500 Data 2014-2015', fontsize=16)

x = list(ret_var['Returns'].dropna().values)
y = list(ret_var['Volatility'].dropna().values)

#print(x, "\n", len(x))
#print(y, "\n", len(y))
#print(z, "\n", len(z))

ax.scatter(x, y, c='r', marker='o')

ax.set_xlabel('Returns')
ax.set_ylabel('Volatility')

plt.show()

In [0]:
#taken from: https://towardsdatascience.com/machine-learning-for-stock-clustering-using-k-means-algorithm-126bc1ace4e1
#https://www.pythonforfinance.net/2018/02/08/stock-clusters-using-k-means-algorithm-in-python/

from pylab import plot,show
from numpy import vstack,array
from numpy.random import rand
import numpy as np
from scipy.cluster.vq import kmeans,vq
import pandas as pd
import pandas_datareader as dr
from math import sqrt
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt



In [0]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

print(ret_var)
#use imputer to remobe NaN values and replace with mean along each column
imp = SimpleImputer(strategy="mean")
scale = StandardScaler()
#use scaler to normalize data before clustering
X = ret_var.values
print(X)

In [0]:
#generate elbow curve
distorsions = []
for k in range(2, 20):
    k_means = KMeans(n_clusters=k)
    k_means.fit(X)
    distorsions.append(k_means.inertia_)

fig = plt.figure(figsize=(15, 5))
plt.plot(range(2, 20), distorsions)
plt.grid(True)
plt.title('Elbow curve')

In [0]:
import  pylab as pl
from sklearn.preprocessing import StandardScaler

#compute K means with n = 5 and get centroids
kmeans = KMeans(n_clusters = 8)
kmeans.fit(X)
centroids = kmeans.cluster_centers_
#print(centroids)
#print(kmeans.labels_)

fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(2,1,1)
fig.suptitle('K-Means(n=5) Clustered S&P 500 Data 2014-2015', fontsize=16)

ax.scatter(x, y, c=k_means.labels_, marker='o')
plot(centroids[:,0],centroids[:,1],'sb',markersize=8)
ax.legend(['Centroids'])
ax.set_xlabel('Returns')
ax.set_ylabel('Volatility')

plt.show()


In [0]:
tickers = pd.DataFrame(ret_var_vol.index, columns=['Tickers'])
cluster_labels = pd.DataFrame(kmeans.labels_,columns=['Cluster'])
labeled_stocks_2d = pd.concat([tickers, cluster_labels],axis = 1).dropna()
print(labeled_stocks_2d)

Now we can compare the cluster labeling between the two analyses to see if adding volume as an additional parameter had a signifigant chnage on the clustering results

In [0]:
#print(labeled_stocks_2d)
#print(labeled_stocks_3d)
print(labeled_stocks_2d[labeled_stocks_2d['Cluster'] != labeled_stocks_3d['Cluster']])